In [ ]:
import os
import statistics
import math
import numpy as np
from ultra_parse import parse_csv
from kalman_filter import KalmanFilter

import matplotlib
#matplotlib.use('Qt5Agg')

from matplotlib import pyplot as plt
plt.rcParams['figure.figsize'] = (15, 5)

# %matplotlib inline

In [ ]:
import import_ipynb
from ultra_mt_filtering import plot_data, linearize_cov, kalman_filter, moving_average

In [ ]:
a45_into_files = []
a45_from_files = []
a45_into_from_files = []
for log_file in os.listdir("logs/"):
    if log_file.startswith("hall_angle_45_into_hall_"):
        a45_into_files.append(log_file)
    elif log_file.startswith("hall_angle_45_from_hall_"):
        a45_from_files.append(log_file)
    elif log_file.startswith("hall_angle_45_into_from_hall_"):
        a45_into_from_files.append(log_file)
        
def parse_and_merge(log_file, all_data):
    data = parse_csv("logs/" + log_file)
    run_number = log_file[len("hall_angle_45_into_hall_"):-len(".csv")]
    for name, values in data.items():
        all_data[f"run_{run_number}"] = values
    return all_data

data_empty = parse_csv("logs/hall_angle_45_empty.csv")

data_into = {}
for into_file in a45_into_files:
    parse_and_merge(into_file, data_into)

data_from = {}
for from_file in a45_from_files:
    parse_and_merge(from_file, data_from)

data_into_from = {}
for if_file in a45_into_from_files:
    parse_and_merge(if_file, data_into_from)

data_ultra_long_empty = parse_csv("ultra_long_empty.csv")

In [ ]:
def threshold_feature(data, threshold, min_feature_time):
    feature_intervals = []
    start_feature = None
    end_feature = None
    for ts, dist in data:
        if dist < threshold:
            if start_feature is None:   # new interval
                start_feature = ts
                end_feature = ts
            else:                       # continue/expand interval
                end_feature = ts
        elif start_feature is not None:
            if end_feature - start_feature >= min_feature_time:
                feature_intervals.append((start_feature, end_feature))
            start_feature = None
            end_feature = None
    if start_feature is not None:
        if end_feature - start_feature >= min_feature_time:
            feature_intervals.append((start_feature, end_feature))
    return feature_intervals

In [ ]:
from PyQt5 import QtWidgets
from matplotlib.backends.backend_qt5agg import FigureCanvasQTAgg as FigureCanvas
from matplotlib.backends.backend_qt5agg import NavigationToolbar2QT as NavigationToolbar

class ScrollableWindow(QtWidgets.QMainWindow):
    def __init__(self, fig):
        self.qapp = QtWidgets.QApplication([])

        QtWidgets.QMainWindow.__init__(self)
        self.widget = QtWidgets.QWidget()
        self.setCentralWidget(self.widget)
        self.widget.setLayout(QtWidgets.QVBoxLayout())
        self.widget.layout().setContentsMargins(0,0,0,0)
        self.widget.layout().setSpacing(0)

        self.fig = fig
        self.canvas = FigureCanvas(self.fig)
        self.canvas.draw()
        self.scroll = QtWidgets.QScrollArea(self.widget)
        self.scroll.setWidget(self.canvas)

        self.nav = NavigationToolbar(self.canvas, self.widget)
        self.widget.layout().addWidget(self.nav)
        self.widget.layout().addWidget(self.scroll)

        self.show()
        exit(self.qapp.exec_()) 

In [ ]:
run = "run_1"
run = "run_hall_three_times_1"
#cur_data = data_into_from[run]
cur_data = data_ultra_long_empty["1"]
data_values = cur_data

#mavg_data = moving_average(data_values)
kalman_dist, kalman_vel, kalman_cov, predict_dist, predict_vel, predict_cov = kalman_filter(data_values, 0.3)
lin_covs = linearize_cov(kalman_cov)
predict_lin_covs = linearize_cov(predict_cov)

kalman_dist_cov00 = [(ts, c[0, 0]+1) for ts, c in kalman_cov]
kalman_dist_cov = [(ts, cov[0]) for ts, cov in lin_covs]
kalman_vel_cov11 = [(ts, c[1,1]+1) for ts, c in kalman_cov]
kalman_vel_cov = [(ts, cov[1]) for ts, cov in lin_covs]
kalman_dist_errs = [(ts, d, kalman_dist_cov[i][1]) for i, (ts, d) in enumerate(kalman_dist)]
kalman_vel_errs = [(ts, v, kalman_vel_cov[i][1]) for i, (ts, v) in enumerate(kalman_vel)]
predict_dist_cov = [(ts, cov[0]) for ts, cov in predict_lin_covs]
predict_vel_cov = [(ts, cov[1]) for ts, cov in predict_lin_covs]
predict_dist_errs = [(ts, d, predict_dist_cov[i][1]) for i, (ts, d) in enumerate(predict_dist)]
predict_vel_errs = [(ts, v, predict_vel_cov[i][1]) for i, (ts, v) in enumerate(predict_vel)]

current_run = {run : cur_data,
               #run + "_moving_average" : mavg_data,
               run + "_kalman_distance" : kalman_dist_errs,
               #run + "kalman_velocity" : kalman_vel,
              }
current_run_vels = {#run : data_from[run],
               #run + "_moving_average" : mavg_data,
               #run + "_kalman_distance" : kalman_dist,
               run + "_kalman_velocity" : kalman_vel_errs,
              }

predict_run = {#run : data_from[run],
               run + "_predict_distance" : predict_dist_errs,
              }
predict_run_vels = {#run : data_from[run],
               run + "_predict_velocity" : predict_vel_errs,
              }
current_run_covs = {#run : data_from[run],
               #run + "_moving_average" : mavg_data,
               run + "_kalman_dist_cov00" : kalman_dist_cov00,
               run + "_kalman_dist_cov" : kalman_dist_cov,
               run + "_kalman_vel_cov11" : kalman_vel_cov11,
               run + "_kalman_vel_cov" : kalman_vel_cov,
              }

In [ ]:
def split_data(data, window_time):
    splits = []
    current_split = []
    for d in data:
        time = d[0]
        if current_split and time - current_split[0][0] > window_time:
            splits.append(current_split)
            current_split = []
        current_split.append(d)
    splits.append(current_split)
    return splits

In [ ]:
n_plot_splits = 10
split_window_time = 20.0
walk_through_threshold = 1.0
walk_through_min_time = 0.2
only_with_features = True

cur_data_split = split_data(cur_data, split_window_time)
kalman_dist_split = split_data(kalman_dist, split_window_time)
kalman_dist_errs_split = split_data(kalman_dist_errs, split_window_time)
print(len(cur_data_split))
print(len(kalman_dist_split))
print(len(kalman_dist_errs_split))
n_plot_splits = min(n_plot_splits, len(cur_data_split))

fig, axes = plt.subplots(n_plot_splits,1, figsize=(15,10*n_plot_splits))
ylim = [0.0, 1.7]
for a in axes:
    a.set_ylim(ylim)

print("Plotting...")

split_index = -1
for i, ax in enumerate(axes):
    if only_with_features:
        split_index += 1
        walk_through = threshold_feature(kalman_dist_split[split_index], walk_through_threshold, walk_through_min_time)
        while not walk_through:
            split_index += 1
            if split_index >= len(kalman_dist_split):
                break
            walk_through = threshold_feature(kalman_dist_split[split_index], walk_through_threshold, walk_through_min_time)
    else:
        split_index = i
        walk_through = threshold_feature(kalman_dist_split[split_index], walk_through_threshold, walk_through_min_time)
    if split_index >= len(cur_data_split):
        break

    run_split = {
        'raw' : cur_data_split[split_index],
        "kalman_dist_errs" : kalman_dist_errs_split[split_index]
    }
    plot_data(ax, run_split, start_at_time_zero=False, label_prefix="45/into_from")
    for start,end in walk_through:
        print(f"At {split_index} - walkthrough: {start} -- {end}")
        ax.fill_betweenx(ylim, start, end, label="walk_through_1m", color="powderblue")
    ax.plot([cur_data_split[split_index][0][0], cur_data_split[split_index][-1][0]], [walk_through_threshold, walk_through_threshold],
            color='red', label='walk_through_threshold')
    ax.xaxis.set_minor_locator(matplotlib.ticker.MultipleLocator(0.1))
    ax.set_title(f"Split {split_index}")
    #ax.legend()